In [ ]:
# Online ayakkabı mağazası olan FLO müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor.
# Buna yönelik olarak müşterilerin davranışlarını tanımlayacak ve bu davranışlardaki öbeklenmelere göre gruplar oluşturulacak.

In [ ]:
# Veri seti Flo'dan son alışverişlerini 2020-2021 yıllarında OmniChannel (hem online hem offline alışveriş yapan)
# olarak yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

In [ ]:
# DEĞİŞKENLER

In [ ]:
# master_id = Eşsiz müşteri numarası
# order_channel = Alışveriş yapılan platform ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile)
# last_order_channel = En son alışveriş yapıldığı kanal
# first_order_date = Müşterinin yaptığı ilk alışveriş tarihi
# last_order_date = Müşterinin yaptığı son alışveriş tarihi
# last_order_date_online = Müşterinin online platformda yaptığı son alışveriş tarihi
# last_order_date_offline = Müşterinin offline platformda yaptığı son alışveriş tarihi
# order_num_total_ever_online = Müşterinin online platformda yaptığı toplam alışveriş sayısı
# order_num_total_ever_offline = Müşterinin offline'da yaptığı toplam alışveriş sayısı
# customer_value_total_ever_offline = Müşterinin offline alışverişlerinde ödediği toplam ücret
# customer_value_total_ever_online = Müşterinin online alışverişlerinde ödediği toplam ücret
# interested_in_categories_12 = Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi

In [ ]:
! pip install lifetimes

In [ ]:
# Gerekli kütüphaneleri içeri aktar
import pandas as pd
import datetime as dt

In [ ]:
pd.set_option("display.max_columns", None)  # Tüm sütunları göster
pd.set_option("display.width",500)    # Sütunları aşağıdan inmeden göster
pd.set_option("display.float_format", lambda x: "%.4f" % x)   # Virgülden sonra 4 basamağa kadar göster

In [ ]:
# Veri setini yüklüyoruz
df_ = pd.read_csv("/content/flo_data_20k.csv")

In [ ]:
df = df_.copy()   # Orijinal veri setini bozmamak için kopyasını alıyoruz

In [ ]:
# Veri setini incele (ilk 10 gözlem)
df.head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR]
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,Desktop,Offline,2018-11-18,2021-03-13,2018-11-18,2021-03-13,1.0000,2.0000,150.8700,49.9900,[KADIN]
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,Android App,Android App,2020-03-04,2020-10-18,2020-10-18,2020-03-04,3.0000,1.0000,59.9900,315.9400,[AKTIFSPOR]
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,Mobile,Offline,2020-05-15,2020-08-12,2020-05-15,2020-08-12,1.0000,1.0000,49.9900,113.6400,[COCUK]
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,Android App,Android App,2020-01-23,2021-03-07,2021-03-07,2020-01-25,3.0000,2.0000,120.4800,934.2100,"[ERKEK, COCUK, KADIN]"
9,1143f032-440d-11ea-8b43-000d3a38a36f,Mobile,Mobile,2019-07-30,2020-10-04,2020-10-04,2019-07-30,1.0000,1.0000,69.9800,95.9800,"[KADIN, AKTIFSPOR]"


In [ ]:
# 'order_channel' sütunundaki farklı kanal değerlerinin sayısını kontrol et
df["order_channel"].value_counts()

,count
order_channel,
Android App,9495
Mobile,4882
Ios App,2833
Desktop,2735


In [ ]:
# 'last_order_channel' sütunundaki farklı kanal değerlerinin sayısını kontrol et
df["last_order_channel"].value_counts()

,count
last_order_channel,
Android App,6783
Offline,6608
Mobile,3172
Ios App,1696
Desktop,1686


In [ ]:
# Veri seti hakkında genel bilgi: Veri tipleri, eksik değerler, vb.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [ ]:
# Veri setinin boyutunu (satır ve sütun sayısını) görüntüle
df.shape

(19945, 12)

In [ ]:
# Eksik (NaN) değerlerin toplamını kontrol et
df.isnull().sum()

,0
master_id,0
order_channel,0
last_order_channel,0
first_order_date,0
last_order_date,0
last_order_date_online,0
last_order_date_offline,0
order_num_total_ever_online,0
order_num_total_ever_offline,0
customer_value_total_ever_offline,0


In [ ]:
# Veri setinin betimsel istatistiklerini göster (ortalama, min, max, vb.)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0000,3.1109,4.2256,1.0000,1.0000,2.0000,4.0000,200.0000
order_num_total_ever_offline,19945.0000,1.9139,2.0629,1.0000,1.0000,1.0000,2.0000,109.0000
customer_value_total_ever_offline,19945.0000,253.9226,301.5329,10.0000,99.9900,179.9800,319.9700,18119.1400
customer_value_total_ever_online,19945.0000,497.3217,832.6019,12.9900,149.9800,286.4600,578.4400,45220.1300


In [ ]:
# Müşterinin toplam alışveriş sayısı
df["Total_Order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
# Müşterinin toplam harcaması
df["Total_Customer"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [ ]:
# Yeni oluşturulan 'Total_Order' ve 'Total_Customer' sütunlarını kontrol et
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,Total_Order,Total_Customer
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]",5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]",2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR],2.0000,209.9800


In [ ]:
# Tarih sütunlarını 'datetime' formatına dönüştür
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.info()    # Değişikliklerin başarılı olup olmadığını kontrol et

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [ ]:
# Alışveriş kanallarına göre müşteri sayısı, toplam alınan ürün sayısı ve toplam harcama dağılımını incele
df.groupby("order_channel").agg({"master_id": "count",
                                 "Total_Order": "sum",
                                 "Total_Customer": "sum"})

,master_id,Total_Order,Total_Customer
order_channel,,,
Android App,9495,52269.0000,7819062.7600
Desktop,2735,10920.0000,1610321.4600
Ios App,2833,15351.0000,2525999.9300
Mobile,4882,21679.0000,3028183.1600


In [ ]:
# En fazla kazancı getiren ilk 10 müşteriyi sıralayalım
df.sort_values("Total_Customer",ascending=False)[:10]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,Total_Order,Total_Customer
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0000,2.0000,684.9700,45220.1300,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.0000,45905.1000
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.0000,1.0000,130.4900,36687.8000,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.0000,36818.2900
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.0000,1.0000,1263.7600,32654.3400,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.0000,33918.1000
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.0000,1.0000,538.9400,30688.4700,"[ERKEK, KADIN, AKTIFSPOR]",11.0000,31227.4100
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.0000,3.0000,18119.1400,2587.2000,[AKTIFSPOR],4.0000,20706.3400
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.0000,4.0000,843.6800,17599.8900,"[ERKEK, KADIN, AKTIFSPOR]",70.0000,18443.5700
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.0000,1.0000,82.4800,16836.0900,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.0000,16918.5700
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.0000,1.0000,180.7300,12545.3700,"[ERKEK, AKTIFSPOR]",37.0000,12726.1000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.0000,1.0000,49.9900,12232.2500,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.0000,12282.2400
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.0000,2.0000,64.9700,12038.1800,"[ERKEK, KADIN]",20.0000,12103.1500


In [ ]:
# En fazla siparişi veren ilk 10 müşteriyi sıralayalım
df.sort_values("Total_Order", ascending=False)[:10]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,Total_Order,Total_Customer
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0000,2.0000,684.9700,45220.1300,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.0000,45905.1000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.0000,1.0000,49.9900,12232.2500,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.0000,12282.2400
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.0000,109.0000,10239.4600,143.9800,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.0000,10383.4400
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.0000,87.0000,8432.2500,139.9800,[ERKEK],88.0000,8572.2300
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.0000,81.0000,3997.5500,242.8100,[ERKEK],83.0000,4240.3600
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.0000,1.0000,1263.7600,32654.3400,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.0000,33918.1000
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.0000,74.0000,4640.7700,543.2800,"[KADIN, AKTIFSPOR]",77.0000,5184.0500
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.0000,3.0000,292.9300,5004.9500,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.0000,5297.8800
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.0000,1.0000,82.4800,16836.0900,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.0000,16918.5700
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.0000,4.0000,843.6800,17599.8900,"[ERKEK, KADIN, AKTIFSPOR]",70.0000,18443.5700


In [ ]:
# Veri ön hazırlık işlemlerini fonksiyon olarak tanımla
def data_preprocessing(dataframe):
  """
    Bu fonksiyon, veri setinde gerekli ön işlemleri yaparak müşteri toplam sipariş sayısı ve toplam harcamasını hesaplar,
    tarih sütunlarını 'datetime' formatına dönüştürür.

    Parametreler:
    dataframe : pandas.DataFrame
        Müşteri verilerini içeren DataFrame.

    Döndürülen Değer:
    pandas.DataFrame
        Veri ön işlemleri yapılmış DataFrame.
    """
  df["Total_Order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
  df["Total_Customer"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
  date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
  dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)
  return dataframe

In [ ]:
# Veri setinin son alışveriş tarihine göre 'Recency', 'Frequency' ve 'Monetary' metriklerini hesapla

In [ ]:
# Recency : Müşterinin son satın alma işleminden bu yana geçen süreyi ifade eder.
# Frequency : Müşterinin belirli bir zaman diliminde kaç kez satın alma yaptığını gösterir.
# Monetary : Müşterinin belirli bir zaman diliminde yaptığı toplam harcama miktarını gösterir.

In [ ]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,Total_Order,Total_Customer
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]",5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]",2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR],2.0000,209.9800


In [ ]:
df["last_order_date"].max()     # Son alışveriş tarihinin max değeri

Timestamp('2021-05-30 00:00:00')

In [ ]:
today_date = dt.datetime(2021, 6, 1)     # Bugünün tarihi olarak kabul et

In [ ]:
rfm = df.groupby("master_id").agg({"last_order_date": lambda last_order_date: (today_date - last_order_date.max()).days,   # Recency: Son alışverişten bugüne kadar geçen gün sayısı
                                   "Total_Order": lambda Total_Order: Total_Order.nunique(),       # Frequency: Kaç kez alışveriş yapıldığı
                                   "Total_Customer": lambda Total_Customer: Total_Customer.sum()})       # Monetary: Toplam harcama miktarı

In [ ]:
rfm.head()

,last_order_date,Total_Order,Total_Customer
master_id,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,1,776.0700
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,1,269.4700
000be838-85df-11ea-a90b-000d3a38a36f,213,1,722.6900
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,1,874.1600
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,1,1620.3300


In [ ]:
# Yeni oluşturduğumuz RFM metriklerine anlamlı isimler verelim
rfm.columns = ["recency", "frequency", "monetary"]

In [ ]:
rfm.head()

,recency,frequency,monetary
master_id,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,1,776.0700
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,1,269.4700
000be838-85df-11ea-a90b-000d3a38a36f,213,1,722.6900
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,1,874.1600
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,1,1620.3300


In [ ]:
# Recency, Frequency ve Monetary metriklerini sıralayarak her biri için skorlar hesapla

In [ ]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])   # Recency skoru: Daha düşük değerler iyi

In [ ]:
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])     # Frequency skoru: Daha yüksek değerler iyi

In [ ]:
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])      # Monetary skoru: Daha yüksek değerler iyi

In [ ]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
master_id,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,1,776.0700,5,1,4
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,1,269.4700,1,1,1
000be838-85df-11ea-a90b-000d3a38a36f,213,1,722.6900,2,1,4
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,1,874.1600,5,1,4
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,1,1620.3300,5,1,5


In [ ]:
# Recency ve Frequency skorlarını birleştirerek RF skoru oluştur
rfm["RF_SCORE"] = (rfm["recency_score"].astype(str)+
                   rfm["frequency_score"].astype(str))

In [ ]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
master_id,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,1,776.0700,5,1,4,51
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,1,269.4700,1,1,1,11
000be838-85df-11ea-a90b-000d3a38a36f,213,1,722.6900,2,1,4,21
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,1,874.1600,5,1,4,51
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,1,1620.3300,5,1,5,51


In [ ]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,19945.0000,134.4584,103.2811,2.0000,43.0000,111.0000,202.0000,367.0000
frequency,19945.0000,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
monetary,19945.0000,751.2443,895.4022,44.9800,339.9800,545.2700,897.7800,45905.1000


In [ ]:
# RF skoru kullanarak farklı müşteri segmentleri tanımlıyoruz

In [ ]:
# Segment isimlendirmeleri için bir eşleşme haritası (seg_map) tanımlayalım
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [ ]:
# RF skorlarını kullanarak müşterileri segmentlere ayıralım
rfm['Segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)

In [ ]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,Segment
master_id,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,1,776.0700,5,1,4,51,new_customers
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,1,269.4700,1,1,1,11,hibernating
000be838-85df-11ea-a90b-000d3a38a36f,213,1,722.6900,2,1,4,21,hibernating
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,1,874.1600,5,1,4,51,new_customers
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,1,1620.3300,5,1,5,51,new_customers


In [ ]:
# Segmentlerin ortalamalarını ve müşteri sayılarını inceleyelim
rfm[["Segment", "recency", "frequency", "monetary"]].groupby("Segment").agg(["mean", "count"])

recency       frequency       monetary      
                        mean count      mean count     mean count
Segment                                                          
about_to_sleep      113.3970  1597    1.0000  1597 696.2609  1597
at_Risk             243.2904  3154    1.0000  3154 652.2101  3154
cant_loose          243.2764  1621    1.0000  1621 663.8024  1621
champions            17.4423  1594    1.0000  1594 921.3811  1594
hibernating         243.9636  3160    1.0000  3160 626.8451  3160
loyal_customers      86.4646  3179    1.0000  3179 784.6352  3179
need_attention      112.6179   780    1.0000   780 716.8077   780
new_customers        17.3895   842    1.0000   842 917.1255   842
potential_loyalists  36.5581  3247    1.0000  3247 880.9118  3247
promising            57.3294   771    1.0000   771 782.1356   771

In [ ]:
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,Segment
master_id,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,1,776.0700,5,1,4,51,new_customers
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,1,269.4700,1,1,1,11,hibernating
000be838-85df-11ea-a90b-000d3a38a36f,213,1,722.6900,2,1,4,21,hibernating
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,1,874.1600,5,1,4,51,new_customers
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,1,1620.3300,5,1,5,51,new_customers


In [ ]:
# Müşteri segmentasyonunu sıfırlayarak 'Segment' sütununu DataFrame'e ekleyelim
rfm = rfm.reset_index()
rfm.head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,Segment
0,00016786-2f5a-11ea-bb80-000d3a38a36f,10,1,776.0700,5,1,4,51,new_customers
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,298,1,269.4700,1,1,1,11,hibernating
2,000be838-85df-11ea-a90b-000d3a38a36f,213,1,722.6900,2,1,4,21,hibernating
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,1,874.1600,5,1,4,51,new_customers
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,1,1620.3300,5,1,5,51,new_customers


In [ ]:
# FLO bünyesine yeni bir kadın ayakkabı markası dahil ediyor. Dahil ettiği markanın ürün fiyatları genel müşteri tercihlerinin üstünde. Bu nedenle markanın
# tanıtımı ve ürün satışları için ilgilenecek profildeki müşterilerle özel olarak iletişime geçeilmek isteniliyor. Bu müşterilerin sadık  ve
# kadın kategorisinden alışveriş yapan kişiler olması planlandı. Müşterilerin id numaralarını csv dosyasına yeni_marka_hedef_müşteri_id.cvs
# olarak kaydediniz.

In [ ]:
target_segments_customer_ids = rfm[rfm["Segment"].isin(["champions","loyal_customers"])]["master_id"]

In [ ]:
cust_ids = df[(df["master_id"].isin(target_segments_customer_ids)) &(df["interested_in_categories_12"].str.contains("KADIN"))]["master_id"]

In [ ]:
cust_ids.to_csv("yeni_marka_hedef_müşteri_id.csv", index=False)

In [ ]:
rfm.head()

,master_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,Segment
0,00016786-2f5a-11ea-bb80-000d3a38a36f,10,1,776.0700,5,1,4,51,new_customers
1,00034aaa-a838-11e9-a2fc-000d3a38a36f,298,1,269.4700,1,1,1,11,hibernating
2,000be838-85df-11ea-a90b-000d3a38a36f,213,1,722.6900,2,1,4,21,hibernating
3,000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,1,874.1600,5,1,4,51,new_customers
4,000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,1,1620.3300,5,1,5,51,new_customers


In [ ]:
# b. Erkek ve Çoçuk ürünlerinde %40'a yakın indirim planlanmaktadır. Bu indirimle ilgili kategorilerle ilgilenen geçmişte iyi müşterilerden olan ama uzun süredir
# alışveriş yapmayan ve yeni gelen müşteriler özel olarak hedef alınmak isteniliyor. Uygun profildeki müşterilerin id'lerini csv dosyasına indirim_hedef_müşteri_ids.csv
# olarak kaydediniz.

In [ ]:
target_segments_customer_ids = rfm[rfm["Segment"].isin(["cant_loose","hibernating","new_customers"])]["master_id"]

In [ ]:
cust_ids = df[(df["master_id"].isin(target_segments_customer_ids)) & ((df["interested_in_categories_12"].str.contains("ERKEK"))|(df["interested_in_categories_12"].str.contains("COCUK")))]["master_id"]

In [ ]:
cust_ids.to_csv("indirim_hedef_müşteri_ids.csv", index=False)

In [ ]:
# Son olarak bu yazdıklarımızı fonksiyon haline getirelim

In [ ]:
def create_rfm(dataframe):
  # Veriyi Hazırlama
  df["Total_Order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
  df["Total_Customer"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]
  date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
  dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)

  # RFM Metriklerinin Hesaplanması
  df["last_order_date"].max()
  today_date = dt.datetime(2021, 6, 1)
  rfm = df.groupby("master_id").agg({"last_order_date": lambda last_order_date: (today_date - last_order_date.max()).days,
                                   "Total_Order": lambda Total_Order: Total_Order.nunique(),
                                   "Total_Customer": lambda Total_Customer: Total_Customer.sum()})
  rfm.columns = ["recency", "frequency", "monetary"]


  # RF Skorlarının Hesaplanması
  rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
  rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
  rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
  rfm["RF_SCORE"] = (rfm["recency_score"].astype(str)+
                   rfm["frequency_score"].astype(str))

  # Segmentlerin İsimlendirilmesi
  seg_map = {
        r'[1-2][1-2]': 'hibernating',
        r'[1-2][3-4]': 'at_Risk',
        r'[1-2]5': 'cant_loose',
        r'3[1-2]': 'about_to_sleep',
        r'33': 'need_attention',
        r'[3-4][4-5]': 'loyal_customers',
        r'41': 'promising',
        r'51': 'new_customers',
        r'[4-5][2-3]': 'potential_loyalists',
        r'5[4-5]': 'champions'
    }

  rfm['Segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)
  rfm = rfm.reset_index()

  return rfm[["master_id", "recency","frequency","monetary","Segment"]]


In [ ]:
def main():
    rfm = create_rfm(df)

    # Yeni kadın markası için hedef müşteri listesi
    target_segments = ["champions", "loyal_customers"]
    target_ids = rfm[rfm["Segment"].isin(target_segments)].index
    target_customers = df[df["master_id"].isin(target_ids) & df["interested_in_categories_12"].str.contains("KADIN")]
    target_customers["master_id"].to_csv("yeni_marka_hedef_müşteri_id.csv", index=False)

    # Erkek ve çocuk indirim kampanyası için hedef müşteri listesi
    discount_segments = ["cant_loose", "hibernating", "new_customers"]
    discount_ids = rfm[rfm["Segment"].isin(discount_segments)].index
    discount_customers = df[df["master_id"].isin(discount_ids) &
                           (df["interested_in_categories_12"].str.contains("ERKEK") |
                            df["interested_in_categories_12"].str.contains("COCUK"))]
    discount_customers["master_id"].to_csv("indirim_hedef_müşteri_ids.csv", index=False)

    print("Segmentasyon tamamlandı ve müşteri listeleri kaydedildi!")

if __name__ == "__main__":
    main()

Segmentasyon tamamlandı ve müşteri listeleri kaydedildi!
